# Portfolio Construction

## Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date

import sys

sys.path.insert(1, "../")


from pathlib import Path

from settings import config


import warnings

from scipy.stats import norm, stats

In [ ]:
from scipy import stats

In [ ]:
from scipy.stats import shapiro, jarque_bera, normaltest, skew, kurtosis

In [ ]:
import plotly.io as pio

pio.templates.default = "plotly_white"
warnings.filterwarnings("ignore")

In [ ]:
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
DATA_DIR = Path(config("DATA_DIR"))
DATA_DIR = DATA_DIR / "options"
WRDS_USERNAME = config("WRDS_USERNAME")

START_DATE_01 = date(1996, 1, 1)
END_DATE_01 = date(2012, 1, 31)

START_DATE_02 = date(2012, 2, 1)
END_DATE_02 = date(2019, 12, 31)

In [ ]:
DATE_RANGE = f"{pd.Timestamp(START_DATE_01):%Y-%m}_{pd.Timestamp(END_DATE_02):%Y-%m}"

## Functions

In [ ]:
# --- Black-Scholes elasticity ---
def bs_elasticity(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(
            d1 - sigma * np.sqrt(T)
        )
        delta = norm.cdf(d1)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d1 + sigma * np.sqrt(T)) - S * norm.cdf(
            -d1
        )
        delta = -norm.cdf(-d1)
    return (delta * S / price), price


# Gaussian kernel function
def kernel_weights(m_grid, ttm_grid, k_s, ttm, bw_m=0.0125, bw_t=10):
    m_grid = np.asarray(m_grid, dtype=float)
    ttm_grid = np.asarray(ttm_grid, dtype=float)
    x = (m_grid - k_s) / bw_m
    y = (ttm_grid - ttm) / bw_t
    dist_sq = x**2 + y**2
    weights = np.exp(-0.5 * dist_sq)
    return weights / weights.sum() if weights.sum() > 0 else np.zeros_like(weights)


# --- Construct a single day portfolio ---
def construct_portfolio(data, k_s_target, ttm_target, option_type="call", r=0.01):
    subset = data[(data["option_type"] == option_type)]
    weights = kernel_weights(subset["moneyness"], subset["ttm"], k_s_target, ttm_target)
    subset = subset.assign(weight=weights)
    subset = subset[subset["weight"] > 0.01]
    subset["weight"] /= subset["weight"].sum()

    # Leverage-adjusted returns
    elast, price = bs_elasticity(
        S=subset["underlying"],
        K=subset["strike"],
        T=subset["ttm"] / 365,
        r=r,
        sigma=subset["iv"],
        option_type=option_type,
    )
    subset["leverage_return"] = subset["daily_return"] / elast

    return (subset["leverage_return"] * subset["weight"]).sum()


# --- Main Loop (simplified) ---
def build_portfolios(option_data, m_grid, ttm_grid, option_types=["call", "put"]):
    portfolios = []
    for opt_type in option_types:
        for k_s in m_grid:
            for ttm in ttm_grid:
                ret = construct_portfolio(option_data, k_s, ttm, option_type=opt_type)
                portfolios.append(
                    {"type": opt_type, "moneyness": k_s, "ttm": ttm, "return": ret}
                )
    return pd.DataFrame(portfolios)


def calc_kernel_weights(spx_mod):
    """Calculate kernel weights for each option in the SPX dataset based on moneyness and maturity targets.
    This function iterates through predefined moneyness and maturity targets, applies kernel weights to candidate options."""

    # Define moneyness and maturity targets from the paper
    moneyness_targets = [0.90, 0.925, 0.950, 0.975, 1.000, 1.025, 1.050, 1.075, 1.100]
    maturity_targets = [30, 60, 90]
    cp_flags = ["C", "P"]

    # Preprocess base DataFrame
    spx_mod["days_to_maturity_int"] = spx_mod["days_to_maturity"].dt.days
    spx_mod = spx_mod.reset_index()
    spx_mod["original_index"] = spx_mod.index

    weight_results = []

    # Iterate through each strategy target
    for cp_flag in cp_flags:
        for target_moneyness in moneyness_targets:
            for target_ttm in maturity_targets:
                # Filter candidate options
                candidate_options = spx_mod[
                    (spx_mod["cp_flag"] == cp_flag)
                    & (spx_mod["moneyness_id"] == target_moneyness)
                    & (spx_mod["maturity_id"] == target_ttm)
                ].copy()

                if candidate_options.empty:
                    continue

                candidate_options["kernel_weight"] = np.nan

                # Apply kernel weights per date
                for date, g in candidate_options.groupby("date"):
                    idx = g.index
                    weights = kernel_weights(
                        g["moneyness"].values,
                        g["days_to_maturity_int"].values,
                        k_s=target_moneyness,
                        ttm=target_ttm,
                    )
                    candidate_options.loc[idx, "kernel_weight"] = weights

                weight_results.append(
                    candidate_options[["original_index", "kernel_weight"]]
                )

    # Merge weights back into spx_mod
    if weight_results:
        all_weights = pd.concat(weight_results).set_index("original_index")
        spx_mod.set_index("original_index", inplace=True)
        spx_mod["kernel_weight"] = all_weights["kernel_weight"]
        spx_mod.reset_index(inplace=True)
    else:
        print("No matching options found for any target.")

    spx_mod.drop(columns=["original_index"], inplace=True)
    return spx_mod

In [ ]:
def calc_option_delta_elasticity(df):
    df = df.copy()

    T = df["days_to_maturity"].dt.days / 365.0
    S = df["close"]
    K = df["strike_price"]
    r = df["tb_m3"] / 100
    sigma = df["IV"]
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

    df = df.assign(
        option_delta=np.where(df["cp_flag"] == "C", norm.cdf(d1), norm.cdf(d1) - 1),
        option_elasticity=lambda x: x["option_delta"] * x["close"] / x["mid_price"],
    )

    return df

In [ ]:
def read_option_data(filename):
    # Example string interval: '(0.9, 0.95]'
    # Remove whitespace and parse the string into tuples
    def parse_interval_string(s):
        # Handle missing or malformed entries gracefully
        if pd.isnull(s) or not isinstance(s, str):
            return pd.NA  # or np.nan
        s = s.strip().replace("(", "").replace("]", "")
        try:
            left, right = map(float, s.split(","))
            return pd.Interval(left, right, closed="right")
        except ValueError:
            return pd.NA

    df = pd.read_parquet(filename)

    # restore the 'moneyness_bin' column as intervals
    df["moneyness_bin"] = df["moneyness_bin"].apply(parse_interval_string)

    return df

In [ ]:
def compute_cjs_return_leverage_investment(spx_mod):
    df = spx_mod.copy()
    df = df.sort_values(["ftfsa_id", "date"])

    # Lag price
    df["mid_price_lag"] = df.groupby("ftfsa_id")["mid_price"].shift(1)

    # Return and daily risk-free rate
    df["option_return"] = (df["mid_price"] - df["mid_price_lag"]) / df["mid_price_lag"]
    df["daily_rf"] = df["tb_m3"] / 100 / 252

    # Weighted dollar investment and return contribution
    df["inv_weight"] = df["kernel_weight"] / df["option_elasticity"]
    df["inv_return"] = df["inv_weight"] * df["option_return"]

    # Group and aggregate
    grouped = df.groupby(["date", "ftfsa_id"])

    port = grouped.agg(
        total_inv_weight=("inv_weight", "sum"),
        total_inv_return=("inv_return", "sum"),
        daily_rf=("daily_rf", "first"),
        cp_flag=("cp_flag", "first"),
    ).reset_index()

    # Apply CJS logic
    def adjusted_return(row):
        if row["cp_flag"] == "C":
            return (
                row["total_inv_return"]
                + (1 - row["total_inv_weight"]) * row["daily_rf"]
            )
        elif row["cp_flag"] == "P":
            return (
                -row["total_inv_return"]
                + (1 + row["total_inv_weight"]) * row["daily_rf"]
            )
        else:
            return np.nan

    port["portfolio_return"] = port.apply(adjusted_return, axis=1)

    return port

In [ ]:
# Function to compute normality metrics
def normality_summary(df):
    summary = []
    for col in df.columns:
        series = df[col].dropna()
        shapiro_p = shapiro(series)[1]
        jb_stat, jb_p = jarque_bera(series)
        normaltest_p = normaltest(series)[1]
        skew_val = skew(series)
        kurt_val = kurtosis(series, fisher=False)
        summary.append(
            {
                "Series": col,
                "Shapiro_p": shapiro_p,
                "JarqueBera_p": jb_p,
                "Normaltest_p": normaltest_p,
                "Skewness": skew_val,
                "Kurtosis": kurt_val,
            }
        )
    return pd.DataFrame(summary)

## Execution

In [ ]:
# read filtered data
source_file = Path(DATA_DIR / f"spx_filtered_final_{DATE_RANGE}.parquet")
spx_filtered = read_option_data(filename=source_file)
spx_filtered = spx_filtered.reset_index()

# create the moneyness ID from the moneyness_bin column, using the right edge of the interval
spx_filtered["moneyness_id"] = spx_filtered["moneyness_bin"].apply(
    lambda x: x.right if pd.notnull(x) else np.nan
)
# drop any rows where moneyness_id is NaN
spx_filtered = spx_filtered.dropna(subset=["moneyness_id"])

spx_filtered

### Construction of Monthly Leverage-Adjusted Portfolio Returns in CJS

**Process**

The construction of the 27 call and 27 put portfolios in CJS is a multi-step process, with the objective of developing portfolio returns series that are stationary and only moderately skewed. Note that the discrete bucketing of moneyness and days to maturity lead to multiple candidate options for each portfolio on each trading day. These options  are given weights according to a **bivariate Gaussian weighting kernel** in moneyness and maturity (bandwidths: *0.0125 in moneyness* and *10 days to maturity*).

Each portfolio's daily returns are initially calculated as simple arithmetic return, assuming the option is bought and sold at its bid-ask midpoint at each rebalancing. The one-day arithmetic return is then converted to a **leverage-adjusted return**. This procedure is achieved by calculating the one-day return of a hypothetical portfolio with $\omega_{BSM}^{-1}$ dollars invested in the option, and $(1 - \omega^{-1})$ dollars invested in the risk-free rate, where $\omega_{BSM}$ is the BSM elasticity based on the implied volatility of the option. 

\begin{align}
\omega_{\text{BSM, Call}} &= \frac{\partial C_{\text{BSM}}}{\partial S} \cdot \frac{S}{C_{\text{BSM}}} > 1 \\
\omega_{\text{BSM, Put}}  &= \frac{\partial P_{\text{BSM}}}{\partial S} \cdot \frac{S}{P_{\text{BSM}}} < -1
\end{align}

Each **leverage-adjusted call portfolio** comprises of a long position in a fraction of a call, and some investment in the risk-free rate. 

Each **leverage-adjusted put portfolio** comprises of a short position in a fraction of a put, and >100% investment in the risk-free rate. 




### Mathematical Details of Portfolio Construction

<font color="blue">*For clarity, we present below the mathematics behind CJS' descriptions of the portfolio construction process. The following applies for a single trading day $t$, for a set of candidate call or put options. Portfolios in CJS are identified by 3 characteristics: option type (call or put), moneyness (9 discrete targets), and time to maturity (3 discrete targets). On any given day, it is rare to find options that exactly match the moneyness and maturity targets. Instead, there may be multiple options that are "close to" the target moneyness / maturity (each a **"candidate option"**). Furthermore, each candidate option has its own price and price sensitivity to changes in the underlying SPX index level. In order to arrive at a "price" for an option portfolio, CJS applies a **Gaussian weighting kernel** in moneyness and maturity, as described below. This kernel-weighted price across the candidate options on a given day is used as the price of the **option component** of the portfolio (the other component being the risk-free rate). This portfolio is leverage-adjusted using the BSM elasticity, in order to standardize the sensitivity of OTM and ITM portfolios to changes in the underlying.*</font>

#### 1. Gaussian Kernel Weighting

Let:

* $m_{i}$ = moneyness of option $i$
* $\tau_{i}$ = days to maturity of option $i$
* $k_{s}$ = target moneyness
* $\tau$ = target maturity
* $h_{m}$, $h_{\tau}$ = bandwidths for moneyness and maturity
* $d_{i}^2 = \left( \frac{m_{i} - k_{s}}{h_{m}} \right)^2 + \left( \frac{\tau_{i} - \tau}{h_{\tau}} \right)^2$

Then the unnormalized Gaussian weight for option $i$ is:

$$
w_{i}^* = \exp\left( -\frac{1}{2} d_{i}^2 \right)
$$

The normalized kernel weight:

$$
w_{i} = \frac{w_{i}^*}{\sum_j w_j^*}
$$

---

#### 2. Option Elasticity

Let:

* $S_{t}$ = underlying index level at time $t$
* $P_{i}$ = price of option $i$
* $\Delta_{i}$ = option delta

Then:

$$
\varepsilon_{i} = \frac{S_t \cdot \Delta_{i}}{P_{i}}
$$

---

#### 3. Arithmetic Return of Option $i$

Let:

* $P_{i,t-1}$ = price of option $i$ at time $t-1$
* $P_{i,t}$ = price of option $i$ at time $t$

Then:

$$
r_{i} = \frac{P_{i,t} - P_{i,t-1}}{P_{i,t-1}}
$$

---

#### 4. Leverage-Adjusted Portfolio Construction

Let:

* $r_{f}$ = risk-free rate on day $t$

The leverage-adjusted return of the call portfolio is:

$$
R_t^{call} = \sum_{i} w_{i} \cdot \frac{1}{\varepsilon_{i}} \cdot r_{i} + \left(1 - \sum_{i} w_{i} \cdot \frac{1}{\varepsilon_{i}} \right) \cdot r_f
$$

The leverage-adjusted return of the put portfolio is:

$$
R_t^{put} = -\sum_{i} w_{i} \cdot \frac{1}{\varepsilon_{i}} \cdot r_{i} + \left(1 + \sum_{i} w_{i} \cdot \frac{1}{\varepsilon_{i}} \right) \cdot r_f
$$



Below we implement this process. 
### Implementation
#### 1. Build the FTSFA ID for each portfolio

In [ ]:
# identify the maturity ID based on the closest maturity to 30, 60, or 90 days
maturity_id = pd.concat(
    (
        abs(spx_filtered["days_to_maturity"].dt.days - 30),
        abs(spx_filtered["days_to_maturity"].dt.days - 60),
        abs(spx_filtered["days_to_maturity"].dt.days - 90),
    ),
    axis=1,
)
maturity_id.columns = [30, 60, 90]
spx_filtered["maturity_id"] = maturity_id.idxmin(axis=1)
spx_filtered["ftfsa_id"] = (
    spx_filtered["cp_flag"]
    + "_"
    + (spx_filtered["moneyness_id"] * 1000).apply(
        lambda x: str(int(x)) if pd.notnull(x) and x == int(x) else str(x)
    )
    + "_"
    + spx_filtered["maturity_id"].astype(str)
)

# set index to ftfsa_id and date
spx_filtered.set_index(["ftfsa_id", "date"], inplace=True)
spx_filtered

In [ ]:
portfolio_ids = spx_filtered.index.get_level_values("ftfsa_id").unique()
portfolio_ids

In [ ]:
spx_filtered["days_to_maturity"].describe()

#### 2. Calculate option elasticity and daily kernel weighting for candidate options for each portfolio. 

In [ ]:
spx_mod = spx_filtered.copy()

# calculate option delta and elasticity
spx_mod = calc_option_delta_elasticity(spx_mod)
# calculate daily kernel weights for candidate options
spx_mod = calc_kernel_weights(spx_mod)
spx_mod

In [ ]:
print("Kernel weight check: All portfolios should sum to 1.0.")
spx_mod.groupby(["date", "ftfsa_id"])["kernel_weight"].sum().round(15).value_counts()

#### 3. Remove options from the portfolio with weights lower than 1%

In [ ]:
spx_mod = spx_mod[spx_mod["kernel_weight"] >= 0.01].reset_index(drop=True)
spx_mod

In [ ]:
# check elasticity > 1 for call options and < -1 for put options
print("Elasticity > 1 for call options?")
spx_mod[spx_mod["cp_flag"] == "C"]["option_elasticity"].describe()["min"] > 1.0

In [ ]:
print("Elasticity < -1 for put options?")
spx_mod[spx_mod["cp_flag"] == "P"]["option_elasticity"].describe()["max"] < -1.0

#### 4. Calculate the daily arithmetic return and the leverage-adjusted return of each portfolio. 

On each trading day, the return of a portfolio is calculated as the <u>weighted average return of the set of candidate options that comprise a single day's option portfolio</u>. The weighting used is the Gaussian kernel weight calculated earlier. Thus the daily return from period $t$ to $t+1$ represents the return from holding a set of candidate options, weighted using the kernel weights as of $t$, from period $t$ to $t+1$. 


In [ ]:
portfolio_returns = compute_cjs_return_leverage_investment(spx_mod)

# Preview result
portfolio_returns.set_index(["date", "ftfsa_id"], inplace=True)

portfolio_returns = portfolio_returns.pivot_table(
    index="date", columns="ftfsa_id", values="portfolio_return"
)
# daily_returns = pd.DataFrame(np.where(portfolio_returns > 1.0, np.nan, portfolio_returns), index=portfolio_returns.index, columns=portfolio_returns.columns)
daily_returns = portfolio_returns.copy()
daily_returns

#### 5. (to be implemented) Filling NaNs
CJS implement an multi-step process to deal with options with missing prices (detailed in section **1.3 Portfolio Formation** of the paper). We reserve the implementation this NaN-filling process for a future version of this dataset. For the current version, we compound the daily portfolio returns into monthly returns, which is the final form of the data utilized in the paper.  

#### 6. Compound Daily Portfolio Returns to Monthly (final 54 portfolios in CJS)

In [ ]:
cjs_returns = daily_returns.resample("M").apply(lambda x: (1 + x).prod() - 1)
cjs_returns = cjs_returns.reset_index().melt(
    id_vars="date", var_name="ftfsa_id", value_name="return"
)
cjs_returns["ftfsa_id"] = "cjs_" + cjs_returns["ftfsa_id"]
cjs_returns = cjs_returns[["ftfsa_id", "date", "return"]].set_index(
    ["ftfsa_id", "date"]
)
# save to data directory
cjs_returns.to_parquet(
    DATA_DIR / f"cjs_portfolio_returns_{DATE_RANGE}.parquet", index=True
)
cjs_returns

## Construction of 18 Portfolio Return Series in He, Kelly, Manela (HKM 2017)

*HKM 2017* reduces the 54 portfolio return series constructed in CJS to 18 by taking an equal-weight average across the 3 maturities for the CJS portfolios with the same moneyness. Below we implement that procedure to obtain the final return series for the FTSFA. 

In [ ]:
hkm_returns = cjs_returns.copy().reset_index()
hkm_returns = hkm_returns.assign(
    type=hkm_returns["ftfsa_id"].apply(lambda x: x.split("_")[1]),
    moneyness_id=hkm_returns["ftfsa_id"].apply(lambda x: x.split("_")[2]),
    maturity_id=hkm_returns["ftfsa_id"].apply(lambda x: x.split("_")[3]),
)
hkm_returns.drop(columns=["ftfsa_id"], inplace=True)
hkm_returns.set_index(["date", "type", "moneyness_id", "maturity_id"], inplace=True)
hkm_returns = hkm_returns.groupby(["date", "type", "moneyness_id"]).mean()
hkm_returns["ftfsa_id"] = (
    "hkm_"
    + hkm_returns.index.get_level_values("type")
    + "_"
    + hkm_returns.index.get_level_values("moneyness_id")
)
hkm_returns = (
    hkm_returns.reset_index()
    .drop(columns=["type", "moneyness_id"])
    .set_index(["ftfsa_id", "date"])
    .sort_index()
)
# save to data directory
hkm_returns.to_parquet(
    DATA_DIR / f"hkm_portfolio_returns_{DATE_RANGE}.parquet", index=True
)
hkm_returns

---

## Data Discussion 

***Note:** Code below is for testing and discussion, not final publication.*

In [ ]:
test_data = hkm_returns.copy()
# test_data = cjs_returns.copy()

test_data = test_data.reset_index().pivot_table(
    columns="ftfsa_id", index="date", values="return"
)

### Tests for Normality of Returns

In [ ]:
returns_df = test_data.copy()

# Apply to returns dataframe
summary_df = normality_summary(returns_df)
display(summary_df.style.format(na_rep="NaN", precision=2))

# Plot histogram and QQ plot for each series
for col in returns_df.columns:
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    sns.histplot(returns_df[col].dropna(), kde=True, stat="density", bins=30, ax=axs[0])
    axs[0].set_title(f"Histogram with KDE: {col}")

    stats.probplot(returns_df[col].dropna(), dist="norm", plot=axs[1])
    axs[1].set_title(f"Q-Q Plot: {col}")

    plt.tight_layout()
    plt.show()

In [ ]:
# plot groups of portfolios as line charts
portfolio_list = []  # ['hkm_C_1000', 'hkm_P_1000'],['cjs_P_950_30', 'cjs_P_1025_60', 'cjs_C_1000_90']

chart_data = test_data[portfolio_list] if portfolio_list else test_data

chart_data.plot(figsize=(16, 6), title="Portfolio Returns Over Time")
plt.xlabel("Date")
plt.xticks(rotation=45)
plt.gca().grid(True, which="major", color="#dddddd", linewidth=0.5)
plt.show()

In [ ]:
metric = "mean"
chart_data.describe().T[metric].plot(
    kind="bar", figsize=(16, 6), title=f"{metric} for leverage adj returns"
)
plt.gca().grid(True, which="major", color="#dddddd", linewidth=0.5)

plt.show()

In [ ]:
(test_data.isna().sum() / test_data.shape[0]).plot(
    kind="bar",
    color="lightblue",
    figsize=(16, 6),
    title="Percentage of Missing Returns by Portfolio",
)
plt.gca().grid(True, which="major", color="#dddddd", linewidth=0.5)
plt.show()

In [ ]:
test_data.skew().plot(kind="bar").set_title("Portfolio Returns Skewness")
plt.xlabel("Portfolio")
plt.xticks(np.arange(len(test_data.columns)), test_data.columns, rotation=90)
plt.gca().grid(True, which="major", color="#dddddd", linewidth=1)
plt.gcf().set_size_inches(16, 6)
plt.ylabel("Skewness")
plt.show()

In [ ]:
test_data.kurtosis().plot(kind="bar").set_title("Portfolio Returns Kurtosis")
plt.xlabel("Portfolio")
plt.xticks(np.arange(len(test_data.columns)), test_data.columns, rotation=90)
plt.gca().grid(True, which="major", color="#dddddd", linewidth=1)
plt.gcf().set_size_inches(16, 6)
plt.ylabel("Kurtosis")
# plt.yscale('log')  # Log scale for better visibility
plt.show()

### Testing NaN-filling logic

In [ ]:
test = spx_mod.set_index(["date", "ftfsa_id"])
test.sort_index(inplace=True)
_cols = ["moneyness", "days_to_maturity_int", "kernel_weight", "exdate"]
test = test[_cols + [_ for _ in test.columns if _ not in _cols]]
test

In [ ]:
daily_returns

In [ ]:
test.loc[pd.IndexSlice["1996-01-04":"1996-01-10", "C_1000_30"], :]

In [ ]:
test.loc[pd.IndexSlice["1996-01-04":"1996-01-10", "C_1025_30"], :]